In [ ]:
import kwant
import numpy as np
import scipy
from matplotlib import pyplot
import kwant.continuum

hamiltonian = "k_x**2 + k_y**2"
template = kwant.continuum.discretize(hamiltonian)
print(template)


energies = []
step=0.021
nsteps=350

fermid0=[]
fermid1=[]
current2=[]
current3=np.zeros((15,8))

mu0=2.51
vbias=0.1
mu1=mu0+vbias
temperature=0.15

for ie in range(nsteps):
    energy = ie * step
    energies.append(energy)
    fd0=kwant.kpm.fermi_distribution(energy, mu0,temperature)
    fd1=kwant.kpm.fermi_distribution(energy, mu1,temperature)
    fermid0.append(fd0)
    fermid1.append(fd1)
    
pyplot.figure()
pyplot.plot(energies, fermid0)
pyplot.plot(energies, fermid1)
pyplot.xlabel("energy [t]")
pyplot.ylabel("Fermi distribution")
pyplot.show()



z0=2
wdt0=2

t=1
a=1
lat = kwant.lattice.square(a,norbs=1)

# Definition of Up lead
sym_up_lead = kwant.TranslationalSymmetry((0, -a))
up_lead = kwant.Builder(sym_up_lead)

for j in range(20):
    up_lead[lat(-23+j,0)] = 4 * t
    if j > 0:
        up_lead[lat(-23+j,0), lat(-23+j-1,0)] = -t
        up_lead[lat(-23+j,1), lat(-23+j,0)] = -t

# Definition of Down lead
sym_down_lead = kwant.TranslationalSymmetry((0, a))
down_lead = kwant.Builder(sym_down_lead)

for j in range(20):
    down_lead[lat(3+j,0)] = 4 * t
    if j > 0:
        down_lead[lat(3+j,0), lat(3+j-1,0)] = -t
        down_lead[lat(3+j,1), lat(3+j,0)] = -t


        
# Construction of 2-parametric family of systems varying smoothness and the width of the third way         
for nz in range(15):
    z=z0+nz*12
    
    # ДВА ПУТИ   
    def twoways(site):
        (x, y) = site.pos
        xl = max(abs(x) - 20, 0)
        xs= max(abs(x) - 28, 0)
        return (1.2*xl**z + 1.2*y**z < 30**z and xs**z + y**z > 14**z) 
    
    syst2 = kwant.Builder()
    syst2.fill(template,twoways, (0, 20));
    syst2.attach_lead(up_lead)
    syst2.attach_lead(down_lead)
     
    if nz==0:
        kwant.plot(syst2)
    
    syst2 = syst2.finalized()
    
    
    data2 = []
    
    for ie in range(nsteps):
        energy = ie * step
        # compute the scattering matrix at a given energy
        smatrix2 = kwant.smatrix(syst2, energy)
        
        # compute the transmission probability from lead 0 to
        # lead 1
        data2.append(smatrix2.transmission(1, 0))
    
    
    prd0=np.multiply(data2,fermid0)
    prd1=np.multiply(data2,fermid1)
    Tcurrent01=scipy.integrate.simps(prd0, energies,even='avg')
    Tcurrent10=scipy.integrate.simps(prd1, energies,even='avg')
    Tcurrent=Tcurrent01-Tcurrent10
    print(Tcurrent)
    current2.append(Tcurrent)
    
    for nw in range(8):
       
    wdt=wdt0+nw*2
        # ТРИ ПУТИ
        def threeways(site):
            (x, y) = site.pos
            xl = max(abs(x) - 20, 0)
            xs= max(abs(x) - 28, 0)
            return (1.2*xl**z + 1.2*y**z < 30**z and xs**z + y**z > 14**z) or (abs(x)<wdt and abs(y)<25.5)
                
        syst3 = kwant.Builder()
        syst3.fill(template,threeways, (0, 10));
        syst3.attach_lead(up_lead)
        syst3.attach_lead(down_lead) 
            
        syst3 = syst3.finalized()
        #kwant.plot(syst3)

        data3 = []

        
        for ie in range(nsteps):
            energy = ie * step
            # compute the scattering matrix at a given energy
            smatrix3 = kwant.smatrix(syst3, energy)
    
            # compute the transmission probability from lead 0 to
            # lead 1
            #energies.append(energy)
            data3.append(smatrix3.transmission(1, 0))



        prd0=np.multiply(data3,fermid0)
        prd1=np.multiply(data3,fermid1)
        Tcurrent01=scipy.integrate.simps(prd0, energies,even='avg')
        Tcurrent10=scipy.integrate.simps(prd1, energies,even='avg')
        Tcurrent=Tcurrent01-Tcurrent10
        print(Tcurrent)
        current3[nz,nw]=Tcurrent    


        pyplot.figure()
        pyplot.plot(energies, data2,color = 'b')
        pyplot.plot(energies, data3, color = 'g')
        pyplot.xlabel("energy [t]")
        pyplot.ylabel("conductance [e^2/h]")
        pyplot.show()

        print(fermid0)
        
        print(current2)
        
        kwant.plot(syst3)
        
        print(current2)
        
        print(current3)